In [2]:
import pandas as pd
import os
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, AffinityPropagation, Birch, MiniBatchKMeans, MeanShift, SpectralClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, pairwise_distances
import datetime as dt
import Funciones_Procesamiento as fp
import rutas

### Importar archivos de consumo

In [13]:
# Ruta de la carpeta que contiene los archivos CSV
ruta_carpeta = rutas.ruta_carpeta_consumos

# Lista para almacenar los DataFrames
dataframes = []

# Iterar sobre cada archivo en la carpeta
for archivo in os.listdir(ruta_carpeta):
    # Verificar si el archivo es un archivo CSV
    if archivo.endswith('.csv'):
        # Construir la ruta completa al archivo
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        # Extraer el nombre del archivo sin la extensión para usar el nombre del cliente en la tabla
        nombre_cliente = archivo.replace('.csv', '')
        # Remover "DATOS" del nombre del cliente
        nombre_cliente = nombre_cliente.replace('DATOS', '')
        # Separar "CLIENTE" del numero de cliente
        nombre_cliente = nombre_cliente.replace('CLIENTE', 'CLIENTE ')
        # Escribir la primera letra en mayúscula
        nombre_cliente = nombre_cliente.title()
        # Leer el archivo CSV en un DataFrame y agregarlo a la lista
        df = pd.read_csv(ruta_completa)
        # Agregar una columna con el nombre del cliente
        df['Cliente'] = nombre_cliente
        dataframes.append(df)

#Observamos y agregamos a una base de datos los sectores económicos de cada cliente
sectores = pd.read_excel('./Datos Electro Dunas/sector_economico_clientes.xlsx')
# Renombrar la columna de la base de datos de sectores
sectores = sectores.rename(columns={'Cliente:':'Cliente'})
# Quitar los espacios en blanco de la columna de cliente
sectores['Cliente'] = sectores['Cliente'].str.strip()

### Preprocesamiento de datos

In [14]:
# Agregar columna de fecha a cada DataFrame
for i, df in enumerate(dataframes):
    dataframes[i] = fp.agregar_col_fecha(df)

# Definición de columnas para el proceso de clustering
mod_cols = ['Cliente', 'Active_energy', 'Reactive_energy', 'Voltaje_FA', 'Horario_laboral', 'Dia_semana']
df_copies = [df.copy() for df in dataframes]
for i, df in enumerate(df_copies):
    df_copies[i] = df_copies[i][mod_cols]

### Entrenamiento de modelos

In [20]:
# Crear dataframe para las métricas de los modelos
metricas = pd.DataFrame(columns=['Cliente', 'Modelo', 'Silhouette', 'Davies Bouldin', 'Calinski Harabasz'])
# Probar diferentes modelos para clustering con k = 2
k = 2
scaler = StandardScaler()
kmeans = KMeans(n_clusters=k)
birch = Birch(n_clusters=k)
spectral = SpectralClustering(n_clusters=k)

modelos = [kmeans, birch, spectral]
nombres = ['KMeans', 'Birch', 'Spectral']
for i, modelo in enumerate(modelos):
    for j, df in enumerate(df_copies):
        cliente = df['Cliente'].iloc[0]
        # Escalar los datos
        X = scaler.fit_transform(df.drop('Cliente', axis=1))
        # Entrenar el modelo
        modelo.fit(X)
        # Agregar columna de cluster al dataframe con el modelo
        dataframes[j]['Cluster_' + nombres[i]] = modelo.labels_
        # Calcular las métricas
        sil = silhouette_score(X, modelo.labels_)
        db = davies_bouldin_score(X, modelo.labels_)
        ch = calinski_harabasz_score(X, modelo.labels_)
        # Agregar las métricas al dataframe
        metricas.loc[len(metricas)] = [cliente, nombres[i], sil, db, ch]

metricas

/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress t

,Cliente,Modelo,Silhouette,Davies Bouldin,Calinski Harabasz
0,Cliente 9,KMeans,0.582220,0.709391,28246.080253
1,Cliente 8,KMeans,0.558982,0.808422,19941.610835
2,Cliente 29,KMeans,0.746514,0.369566,6785.038766
3,Cliente 15,KMeans,0.465073,0.929267,14439.141486
4,Cliente 14,KMeans,0.598323,0.614947,8047.389046
...,...,...,...,...,...
85,Cliente 2,Spectral,0.637408,0.581312,24783.916044
86,Cliente 6,Spectral,0.601940,0.674182,22736.049258
87,Cliente 7,Spectral,0.635886,0.606366,23706.877127
88,Cliente 5,Spectral,0.651734,0.532664,11073.995736


In [17]:
metricas.to_csv('metricas_modelos_clusterv4.csv', index=False)

### Comparación de modelos

In [10]:
# Para cada cliente, encontrar el mejor modelo. 
# El mejor modelo es el que tiene el silhouette score más alto, el davies bouldin score más bajo y el calinski harabasz score más alto, en ese orden de prioridad. 
# Crear dataframe para almacenar el mejor modelo para cada cliente
mejores_modelos = pd.DataFrame(columns=['Cliente', 'Modelo'])
for i in range(1, 31):
    cliente = 'Cliente ' + str(i)
    df_cliente = metricas[metricas['Cliente'] == cliente]
    df_cliente = df_cliente.sort_values(by=['Silhouette', 'Davies Bouldin', 'Calinski Harabasz'], ascending=[False, True, False])
    mejores_modelos.loc[len(mejores_modelos)] = [cliente, df_cliente.iloc[0]['Modelo']]
mejores_modelos


,Cliente,Modelo
0,Cliente 1,KMeans
1,Cliente 2,Spectral
2,Cliente 3,Birch
3,Cliente 4,KMeans
4,Cliente 5,Birch
5,Cliente 6,Birch
6,Cliente 7,Spectral
7,Cliente 8,KMeans
8,Cliente 9,Birch
9,Cliente 10,KMeans


In [11]:
# Evaluar el mejor modelo promediando las métricas de todos los clientes
resumen_mejores = metricas.drop('Cliente', axis=1).groupby('Modelo').mean()
display(resumen_mejores)

# Graficar las métricas de los mejores modelos, incluir las 3 barras con diferente color para cada métrica
fig = px.bar(resumen_mejores.reset_index(), x='Modelo', y=['Silhouette', 'Davies Bouldin', 'Calinski Harabasz'], title='Métricas de los mejores modelos', barmode='group')
fig.show()

,Silhouette,Davies Bouldin,Calinski Harabasz
Modelo,,,
Birch,0.385483,1.214171,5015.100920
KMeans,0.348121,1.368415,5951.955889
Spectral,0.404314,1.124750,4475.338547


### Ver histórico clusterizado de cada cliente

In [26]:
# Revisar para cada dataframe en dataframes la media de la energía activa. La mayor media será "anomalia", la menor será "normal"
cluster_cols = ['Cluster_KMeans', 'Cluster_Birch', 'Cluster_Spectral']
for i, df in enumerate(dataframes):
    for col in cluster_cols:
        ae_means = dataframes[i][[col, 'Active_energy']].groupby(col).mean()
        if ae_means.loc[0]['Active_energy'] > ae_means.loc[1]['Active_energy']:
            anomalia = 0
            normal = 1
        else:
            anomalia = 1
            normal = 0
        # Agregar columna de anomalia al dataframe
        dataframes[i]['Anomalia_'+col] = dataframes[i][col].apply(lambda x: 'Anomalia' if x == anomalia else 'Normal')

In [27]:
df_consolidado = pd.concat(dataframes)
df_consolidado.to_csv('df_consolidado_clusterv4.csv', index=False)

In [75]:
X = scaler.fit_transform(df_copies[23][mod_cols].drop('Cliente', axis=1))
kmeans.fit(X)
dataframes[23]['Cluster_KMeans'] = kmeans.labels_
fig = px.scatter(dataframes[23], x='Fecha', y='Active_energy', color='Cluster_KMeans', title=dataframes[23]['Cliente'].iloc[0])
fig.show()

/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [76]:
X = scaler.fit_transform(df_copies[23][mod_cols].drop('Cliente', axis=1))
birch.fit(X)
dataframes[23]['Cluster_Birch'] = birch.labels_
fig = px.scatter(dataframes[23], x='Fecha', y='Active_energy', color='Cluster_Birch', title=dataframes[23]['Cliente'].iloc[0])
fig.show()

In [77]:
X = scaler.fit_transform(df_copies[23][mod_cols].drop('Cliente', axis=1))
spectral.fit(X)
dataframes[23]['Cluster_Spectral'] = spectral.labels_
fig = px.scatter(dataframes[23], x='Fecha', y='Active_energy', color='Cluster_Spectral', title=dataframes[23]['Cliente'].iloc[0])
fig.show()

In [79]:
# Correr modelo kmeans para todos los dataframes
for i, df in enumerate(df_copies):
    X = scaler.fit_transform(df.drop('Cliente', axis=1))
    kmeans.fit(X)
    dataframes[i]['Cluster_KMeans'] = kmeans.labels_

# Revisar para cada dataframe en dataframes la media de la energía activa. La mayor media será "anomalia", la menor será "normal"
cluster_cols = ['Cluster_KMeans', 'Cluster_Birch', 'Cluster_Spectral']
for i, df in enumerate(dataframes):
    for col in cluster_cols:
        ae_means = dataframes[i][[col, 'Active_energy']].groupby(col).mean()
        if ae_means.loc[0]['Active_energy'] > ae_means.loc[1]['Active_energy']:
            anomalia = 0
            normal = 1
        else:
            anomalia = 1
            normal = 0
        # Agregar columna de anomalia al dataframe
        dataframes[i]['Anomalia_'+col] = dataframes[i][col].apply(lambda x: 'Anomalia' if x == anomalia else 'Normal')

df_consolidado = pd.concat(dataframes)
df_consolidado.to_csv('df_consolidado_clusterv5.csv', index=False, encoding='latin1')

/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/gabrielgomez/Library/Pyth